In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from graphviz import Digraph

# Função para criar um exemplo de AFD (Autômato Finito Determinístico)
def criar_afd():
    """Cria um exemplo de AFD para ser minimizado."""
    afd = {
        'estados': {'q0', 'q1', 'q2', 'q3', 'q4', 'q5'},  # Conjunto de estados
        'alfabeto': {'a', 'b'},  # Alfabeto aceito pelo AFD
        'transicoes': {  # Função de transição (estado atual, símbolo) -> próximo estado
            ('q0', 'b'): 'q1', ('q0', 'a'): 'q2',
            ('q1', 'a'): 'q1', ('q1', 'b'): 'q0',
            ('q2', 'a'): 'q4', ('q2', 'b'): 'q5',
            ('q3', 'a'): 'q5', ('q3', 'b'): 'q4',
            ('q4', 'a'): 'q3', ('q4', 'b'): 'q2',
            ('q5', 'a'): 'q2', ('q5', 'b'): 'q3'
        },
        'inicio': 'q0',  # Estado inicial
        'aceite': {'q0', 'q4', 'q5'},  # Conjunto de estados de aceitação
    }
    return afd

# Função para minimizar o AFD usando o algoritmo de Hopcroft
def minimizacao_hopcroft(afd):
    """Minimiza o AFD usando o algoritmo de Hopcroft."""
    estados = afd['estados']
    alfabeto = afd['alfabeto']
    transicoes = afd['transicoes']
    aceite_estados = afd['aceite']
    nao_aceite_estados = estados - aceite_estados  # Estados não finais

    # Inicialização das partições
    P = [aceite_estados, nao_aceite_estados]  # Conjunto de estados particionado
    Q = [aceite_estados] if aceite_estados else []  # Fila de partições a serem analisadas

    # Loop principal do algoritmo
    while Q:
        A = Q.pop()  # Seleciona um conjunto para análise
        for c in alfabeto:  # Para cada símbolo do alfabeto
            X = {s for s in estados if transicoes.get((s, c)) in A}  # Estados que transitam para A com c
            novo_P = []

            for Y in P:  # Divide cada partição de acordo com a transição
                intersecao = X & Y
                diferenca = Y - X

                if intersecao and diferenca:  # Se a partição puder ser dividida
                    novo_P.append(intersecao)
                    novo_P.append(diferenca)

                    # Atualiza a fila Q
                    if Y in Q:
                        Q.remove(Y)
                        Q.append(intersecao)
                        Q.append(diferenca)
                    else:
                        Q.append(intersecao if len(intersecao) <= len(diferenca) else diferenca)
                else:
                    novo_P.append(Y)

            P = novo_P  # Atualiza a partição principal

    # Reconstrói o AFD minimizado
    minimizado = {}
    for grupo in P:  # Mapeia os estados para seus representantes
        rep = next(iter(grupo))  # Representante do grupo
        for estado in grupo:
            minimizado[estado] = rep

    # Cria os novos componentes do AFD
    min_estados = set(minimizado.values())  # Estados reduzidos
    min_transicoes = {}
    for (estado, simbolo), alvo in transicoes.items():
        if estado in minimizado and alvo in minimizado:
            min_transicoes[(minimizado[estado], simbolo)] = minimizado[alvo]

    min_aceite = {minimizado[s] for s in aceite_estados}  # Estados de aceitação minimizados
    min_inicio = minimizado[afd['inicio']]  # Estado inicial minimizado

    # Retorna o AFD minimizado
    afd_minimizado = {
        'estados': min_estados,
        'alfabeto': alfabeto,
        'transicoes': min_transicoes,
        'inicio': min_inicio,
        'aceite': min_aceite,
    }

    return afd_minimizado

# Função para testar se uma palavra é aceita por um AFD
def teste_palavra(afd, palavra):
    """Testa se uma palavra é aceita pelo AFD."""
    current_state = afd['inicio']  # Começa no estado inicial
    for symbol in palavra:
        current_state = afd['transicoes'].get((current_state, symbol))  # Transita para o próximo estado
        if current_state is None:  # Se não houver transição, rejeita a palavra
            return False
    return current_state in afd['aceite']  # Verifica se o estado final é de aceitação

# Função para comparar dois AFDs com base nas palavras aceitas
def compare_afds(afd1, afd2, palavras):
    """Compara se as palavras aceitas pelo AFD1 são aceitas pelo AFD2."""
    resultados = {}
    for palavra in palavras:
        resultado1 = teste_palavra(afd1, palavra)  # Resultado no AFD original
        resultado2 = teste_palavra(afd2, palavra)  # Resultado no AFD minimizado
        resultados[palavra] = (resultado1, resultado2)
    return resultados

# Criação do AFD inicial
afd = criar_afd()

# Minimização do AFD
afd_minimizado = minimizacao_hopcroft(afd)
print(afd_minimizado)
# Função para criar um grafo visual do AFD
def criar_grafo_afd(afd, nome):
    dot = Digraph(nome, format='png')  # Inicializa o grafo
    dot.attr(rankdir='LR')  # Direção da leitura do AFD
    for estado in afd['estados']:
        if estado in afd['aceite']:
            dot.node(estado, shape='doublecircle')  # Estados de aceitação
        else:
            dot.node(estado, shape='circle')  # Estados normais
    dot.node('', shape='none')  # Estado vazio para a seta inicial
    dot.edge('', afd['inicio'])  # Seta para o estado inicial
    for (estado_inicial, simbolo), estado_final in afd['transicoes'].items():
        dot.edge(estado_inicial, estado_final, label=simbolo)  # Transições
    return dot

# Criação dos gráficos para os AFDs
grafo_afd1 = criar_grafo_afd(afd, 'AFD1')
grafo_afd2 = criar_grafo_afd(afd_minimizado, 'AFD2')

# Exibe os gráficos gerados
grafo_afd1.view(cleanup=True)
grafo_afd2.view(cleanup=True)

# Testa algumas palavras nos AFDs
palavras_para_teste = ["b", "a", "aa", "ab", "ba", "bbb"]
comparison_results = compare_afds(afd, afd_minimizado, palavras_para_teste)

# Mostra os resultados da comparação
print("Comparação do aceite de diferentes palavras entre os AFDs:")
for palavra, (resultado1, resultado2) in comparison_results.items():
    print(f"Palavra '{palavra}': Original AFD = {resultado1}, AFD minimizado = {resultado2}")


{'estados': {'q4', 'q1', 'q2', 'q0'}, 'alfabeto': {'a', 'b'}, 'transicoes': {('q0', 'b'): 'q1', ('q0', 'a'): 'q2', ('q1', 'a'): 'q1', ('q1', 'b'): 'q0', ('q2', 'a'): 'q4', ('q2', 'b'): 'q4', ('q4', 'a'): 'q2', ('q4', 'b'): 'q2'}, 'inicio': 'q0', 'aceite': {'q4', 'q0'}}
Comparação do aceite de diferentes palavras entre os AFDs:
Palavra 'b': Original AFD = False, AFD minimizado = False
Palavra 'a': Original AFD = False, AFD minimizado = False
Palavra 'aa': Original AFD = True, AFD minimizado = True
Palavra 'ab': Original AFD = True, AFD minimizado = True
Palavra 'ba': Original AFD = False, AFD minimizado = False
Palavra 'bbb': Original AFD = False, AFD minimizado = False


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from graphviz import Digraph

def criar_afd():
    """Cria um exemplo de AFD para ser minimizado."""
    afd = {
        'estados': {'q0', 'q1', 'q2', 'q3', 'q4'},
        'alfabeto': {'0', '1'},
        'transicoes': {
            ('q0', '0'): 'q1', ('q0', '1'): 'q3',
            ('q1', '0'): 'q1', ('q1', '1'): 'q2',
            ('q2', '0'): 'q3', ('q2', '1'): 'q4',
            ('q3', '0'): 'q3', ('q3', '1'): 'q4',
            ('q4', '0'): 'q1', ('q4', '1'): 'q2'
        },
        'inicio': 'q0',
        'aceite': {'q2','q4'},
    }
    return afd

# Criação do AFD inicial
afd = criar_afd()

# Minimização do AFD
afd_minimizado = minimizacao_hopcroft(afd)

# Criar os gráficos para os dois AFDs
grafo_afd1 = criar_grafo_afd(afd, 'AFD1')
grafo_afd2 = criar_grafo_afd(afd_minimizado, 'AFD2')

# Exibir os dois gráficos
grafo_afd1.view(cleanup=True)
grafo_afd2.view(cleanup=True)


# Teste de palavras
palavras_para_teste = ["b", "a", "aa", "ab", "ba", "bbb"]
comparison_results = compare_afds(afd, afd_minimizado, palavras_para_teste)

print("Comparação do aceite de diferentes palavras entre os AFDs:")
for palavra, (resultado1, resultado2) in comparison_results.items():
    print(f"Palavra '{palavra}': Original AFD = {resultado1}, AFD minimizado = {resultado2}")

Comparação do aceite de diferentes palavras entre os AFDs:
Palavra 'b': Original AFD = False, AFD minimizado = False
Palavra 'a': Original AFD = False, AFD minimizado = False
Palavra 'aa': Original AFD = False, AFD minimizado = False
Palavra 'ab': Original AFD = False, AFD minimizado = False
Palavra 'ba': Original AFD = False, AFD minimizado = False
Palavra 'bbb': Original AFD = False, AFD minimizado = False
